In [1]:
import pandas as pd
import numpy as np
import pymrio

In [2]:
io = pymrio.parse_exiobase3('C://Users/Maxime/Desktop/Thesis/Bdd/IOT_2011_pxp/')

In [3]:
io.calc_all()

In [4]:
# emissions in $ and not in M$
io.satellite.S.loc[[i for i in io.satellite.S.index if io.satellite.unit.loc[i,'unit'] not in ['M.EUR','1000 p','M.hr']]]=io.satellite.S.loc[[i for i in io.satellite.S.index if io.satellite.unit.loc[i,'unit'] not in ['M.EUR','1000 p','M.hr']]]/1000000

In [5]:
S = io.satellite.S.copy()
S = pd.concat([S,S.loc[:,[i for i in io.satellite.S.columns if i[0] == 'CA']]],axis=1)
countries = [i[0] for i in S.iloc[:,::200].columns]
countries[-1]='QC'
S.columns = pd.MultiIndex.from_product([countries,[i[1] for i in S.iloc[:,:200].columns]],names=('region', 'sector'))
S[('QC','grid_mix')] = pd.Series(0, S.index)

In [6]:
C = pd.concat([pd.read_excel(
                'C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/characterisationEXIOBASE3_adaptedFromEXIOBASE2.xlsx',
                'Q_emission'),
                pd.read_excel(
                'C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/characterisationEXIOBASE3_adaptedFromEXIOBASE2.xlsx',
                    'Q_materials'),
                pd.read_excel(
                'C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/characterisationEXIOBASE3_adaptedFromEXIOBASE2.xlsx',
                    'Q_resources'),
                pd.read_excel(
                'C://Users/Maxime/Desktop/Thesis/Modules_Python/pylcaio/src/Data/characterisationEXIOBASE3_adaptedFromEXIOBASE2.xlsx',
                    'Q_factor_inputs')], sort=False).fillna(0)

In [7]:
with open('technology_matrix.pickle','rb') as f:
    A = pd.read_pickle(f)
    
with open('final_demand_1st_quintile.pickle','rb') as f:
    Y = pd.read_pickle(f)
other_quintiles = ['2nd_quintile','3rd_quintile','4th_quintile','5th_quintile']
for quintile in other_quintiles:
    with open('final_demand_'+str(quintile)+'.pickle','rb') as f:
        df = pd.read_pickle(f)
    Y = pd.concat([Y, df],axis=1)
Y.columns = ['1st_quintile','2nd_quintile','3rd_quintile','4th_quintile','5th_quintile']

In [8]:
I = pd.DataFrame(np.eye(len(A)),A.index,A.columns)
X = pd.DataFrame(np.linalg.solve(I-A,Y),A.index, Y.columns)

In [9]:
indirect_emissions = S.dot(X)
direct_emissions = (pd.DataFrame(io.satellite.FY.loc[:, ('CA','Final consumption expenditure by households')])/
                    io.Y.loc[:,('CA','Final consumption expenditure by households')].sum()/1000000)
direct_emissions = pd.concat([direct_emissions]*5,axis=1)
direct_emissions.columns = Y.columns
direct_emissions *= Y.sum()
total_emissions = indirect_emissions + direct_emissions

In [10]:
GWP_per_quintile_per_household = pd.DataFrame(C.dot(total_emissions).iloc[5]*0.72)

In [11]:
GWP_per_quintile_per_household

,"Problem oriented approach: baseline (CML, 2001)"
,global warming (GWP100)
,"GWP100 (IPCC, 2007)"
,kg CO2 eq.
1st_quintile,11913.104338
2nd_quintile,18067.072252
3rd_quintile,25122.874704
4th_quintile,32224.358209
5th_quintile,46382.813640


In [12]:
GWP_per_quintile_per_dollar = pd.DataFrame(C.dot(total_emissions).iloc[5]*0.72) / pd.DataFrame(Y.sum()).values

In [13]:
GWP_per_quintile_per_dollar

,"Problem oriented approach: baseline (CML, 2001)"
,global warming (GWP100)
,"GWP100 (IPCC, 2007)"
,kg CO2 eq.
1st_quintile,0.514747
2nd_quintile,0.552483
3rd_quintile,0.542282
4th_quintile,0.558689
5th_quintile,0.559020


In [14]:
GWP_quebec_total = GWP_per_quintile_per_household.sum()[0]*3395345/5

In [15]:
print(str(int(GWP_quebec_total/1000000000))+'MtCO2eq')

90MtCO2eq


In [16]:
print(str(int(GWP_quebec_total/1000000000/8.4))+'tCO2eq/habitant')

10tCO2eq/habitant


In [17]:
print(str(int(GWP_quebec_total/1000000000/360*100))+"% of Canada's households footprint")
print(str(int(GWP_quebec_total/1000000000/590*100))+"% of Canada's total footprint")

25% of Canada's households footprint
15% of Canada's total footprint


### Purchases from households

In [ ]:
# for each Quebec's quintile
(Y.groupby(level='sector').sum()/Y.groupby(level='sector').sum().sum()*100).to_excel('C://Users/Maxime/Desktop/Thesis/Excel/Save.xlsx')

In [ ]:
# for Quebec as a whole
(Y.groupby(level='sector').sum().sum(axis=1)*3395345/5).to_excel('C://Users/Maxime/Desktop/Thesis/Excel/Save.xlsx')

In [ ]:
# for Canada
(io.Y.loc[:,('CA','Final consumption expenditure by households')].groupby(level='sector').sum()*1000000).to_excel('C://Users/Maxime/Desktop/Thesis/Excel/Save.xlsx')

### Contribution analysis

In [ ]:
# per quintile
quintile = '5th_quintile'
I = pd.DataFrame(np.eye(len(A)),A.index,A.columns)
diag_Y = pd.DataFrame(np.diagflat(Y.loc[:,quintile].values*0.72),A.index,A.columns)
X = pd.DataFrame(np.linalg.solve(I-A,diag_Y),A.index, diag_Y.columns)
D = C.dot(S).dot(X)
boo = pd.DataFrame(D.iloc[5])
boo.index = boo.index.values
boo.to_excel('C://Users/Maxime/Desktop/Thesis/Excel/Save.xlsx')

In [133]:
# for Quebec as a whole
I = pd.DataFrame(np.eye(len(A)),A.index,A.columns)
diag_Y = pd.DataFrame(np.diagflat((Y*3395345/5*0.72).sum(axis=1).values),A.index,A.columns)
X = pd.DataFrame(np.linalg.solve(I-A,diag_Y),A.index, diag_Y.columns)
D = C.dot(S).dot(X)
boo = pd.DataFrame(D.iloc[5])
boo.index = boo.index.values
boo.to_excel('C://Users/Maxime/Desktop/Thesis/Excel/Save.xlsx')

In [ ]:
# for Canada
I = pd.DataFrame(np.eye(len(io.A)),io.A.index,io.A.columns)
diag_Y = pd.DataFrame(np.diagflat(io.Y.loc[:,('CA','Final consumption expenditure by households')].values),io.A.index,io.A.columns)
X = pd.DataFrame(np.linalg.solve(I-io.A,diag_Y*1000000),io.A.index, diag_Y.columns)
D = C.dot(io.satellite.S).dot(X)
boo = pd.DataFrame(D.iloc[5])
boo.index = boo.index.values
boo.to_excel('C://Users/Maxime/Desktop/Thesis/Excel/Save.xlsx')

### Contribution analyses with higher level classification

In [81]:
classification = pd.read_excel('C://Users/Maxime/Desktop/Thesis/BD_conso_QC/Concordance_high_level.xlsx','Sheet2')

In [82]:
from collections import defaultdict
divide_by_wat = defaultdict(list)

for index in classification.index:
    for column in classification.columns[2:]:
        if classification.loc[index, 'Vision consommateur détaillée'] not in divide_by_wat[classification.loc[index,column]]:
            divide_by_wat[classification.loc[index,column]].append(classification.loc[index, 'Vision consommateur détaillée'])
del divide_by_wat[np.nan]

In [132]:
# per quintile
quintile = '5th_quintile'
I = pd.DataFrame(np.eye(len(A)),A.index,A.columns)
diag_Y = pd.DataFrame(np.diagflat(Y.loc[:,quintile].values*0.72),A.index,A.columns)
X = pd.DataFrame(np.linalg.solve(I-A,diag_Y),A.index, diag_Y.columns)
D = C.dot(S).dot(X)
df = pd.DataFrame(D.iloc[5])

In [139]:
# for Quebec as a whole
I = pd.DataFrame(np.eye(len(A)),A.index,A.columns)
diag_Y = pd.DataFrame(np.diagflat((Y*3395345/5*0.72).sum(axis=1).values),A.index,A.columns)
X = pd.DataFrame(np.linalg.solve(I-A,diag_Y),A.index, diag_Y.columns)
D = C.dot(S).dot(X)
df = pd.DataFrame(D.iloc[5])

In [140]:
reclassified_df = df.copy()
for index in reclassified_df.index:
    if len(divide_by_wat[index[1]]) != 0:
        reclassified_df.loc[index,:] /= len(divide_by_wat[index[1]])

In [141]:
reclassified = pd.DataFrame(0,index=pd.MultiIndex.from_product([Y.index.levels[0],classification.loc[:,'Vision consommateur détaillée']]),columns=df.columns)
for index in reclassified.index:
    for sector in divide_by_wat.keys():
        if sector == 'grid_mix' and index[0] != 'QC':
            pass
        else:
            if index[1] in divide_by_wat[sector]:
                reclassified.loc[index,:] += reclassified_df.loc[(index[0],sector)]

In [142]:
reclassified = reclassified.groupby([i[1] for i in reclassified.index]).sum()

In [143]:
for index in reclassified.index:
    for vision_consommateur in classification.index:
        if classification.loc[vision_consommateur,'Vision consommateur détaillée'] == index:
            reclassified.loc[index,'higher level'] = classification.loc[vision_consommateur,'Vision consommateur']

In [144]:
def add_public_services(population_canada, personne_par_menage):
    # on assume une responsabilite des services sociaux partages equitablement a travers tous les canadiens! (c'est faux bien entendu)
    result = C.dot(io.satellite.S).dot(io.Y)*1000000
    return result.loc[[('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')],('CA','Final consumption expenditure by government')][0]/population_canada*personne_par_menage

reclassified.loc['Services administratifs et gouvernementaux',
                 ('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')] = add_public_services(37000000,2.3)

In [138]:
# for each quintile, redistribute direct emissions
reclassified.loc['Transport quotidien',
                 ('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')] += 0.8 * C.dot(direct_emissions).loc[[('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')],quintile][0]
reclassified.loc['Chauffage/Eclairage/Eau chaude',
                 ('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')] += 0.2 * C.dot(direct_emissions).loc[[('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')],quintile][0]
reclassified.to_excel('C://Users/Maxime/Desktop/Thesis/Excel/Save.xlsx')

In [145]:
# for whole Quebec, redistribute direct emissions
reclassified.loc['Transport quotidien',
                 ('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')] += 0.8 * C.dot(direct_emissions).loc[[('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')]].sum().sum()
reclassified.loc['Chauffage/Eclairage/Eau chaude',
                 ('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')] += 0.2 * C.dot(direct_emissions).loc[[('Problem oriented approach: baseline (CML, 2001)',
                  'global warming (GWP100)',
                  'GWP100 (IPCC, 2007)',
                  'kg CO2 eq.')]].sum().sum()
reclassified.to_excel('C://Users/Maxime/Desktop/Thesis/Excel/Save.xlsx')